# Develop Solar Plant Optimization Problem using Excel Interface

Test classes and functions from `optprob` module for comparing different optimization algorithms.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import skopt

import xlwings as xw

from problems.optprob.problems import (
    ConstrainedScalarOptimizationProblem,
    solve_problem_with_optimizer, 
    solve_problem_with_optimizer_n_repeats
)
from problems.optprob.plot_utils import (
    function_evaluations_plot, 
    best_guesses_plot, 
    best_guesses_plot_n_repeats
)

from excel_tools.run_excel_sheet_mac import (
    get_var_value, 
    set_var_value, 
    evaluate_excel_sheet
)

from platform import python_version
python_version()

In [ ]:
import lpfgopt
lpfgopt.__version__

## Optimization Problem Definition

In [ ]:
# Path to Excel file
problems_dir = 'problems'
problem_name = 'solar_plant_rto'
excel_filename = 'Solar Plant Optimization of N-Pumps I-O 2025-08-29.xlsm'
filepath = os.path.join(os.getcwd(), problems_dir, problem_name, excel_filename)

# Check if file exists
if not os.path.exists(filepath):
    raise FileNotFoundError(f"Excel file not found: {filepath}")

In [ ]:
# Open the workbook with xlwings
wb = xw.Book(filepath)

cell_refs = {
    'name': ((2, 2), 'C2'),
    'f(x)': ((6, 13), (6, 14)),
    'x': ((5, 8), [(6, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8)]),
    'g(x)': None,
    'x_lb': ((5, 9), [(6, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), (16, 9), (17, 9), (18, 9), (19, 9), (20, 9), (21, 9), (22, 9)]),
    'x_ub': ((5, 10), [(6, 10), (9, 10), (10, 10), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), (16, 10), (17, 10), (18, 10), (19, 10), (20, 10), (21, 10), (22, 10)])
}

# cell_refs = {
#     'name': ('B2', 'C2'),
#     'f(x)': ('M6', 'N6'),
#     'x': ('H5', [(6, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8)]),
#     #'g(x)': None,
#     'x_lb': ('I5', [(6, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), (16, 9), (17, 9), (18, 9), (19, 9), (20, 9), (21, 9), (22, 9)]),
#     'x_ub': ('J5', [(6, 10), (9, 10), (10, 10), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), (16, 10), (17, 10), (18, 10), (19, 10), (20, 10), (21, 10), (22, 10)])
# }

inputs = {'x': [
    0.8,
    0.917004705,
    0.886780974,
    0.853623916,
    0.826111695,
    0.801142972,
    0.783225649,
    0.760051185,
    0.900587117,
    0.884820289,
    0.85561633,
    0.827130134,
    0.797646554,
    0.78549898,
    0.755400025
]}

try:
    outputs = evaluate_excel_sheet(
        wb, inputs, cell_refs, output_vars=['name', 'f(x)', 'x_lb', 'x_ub']
    )
    print("Test: ", outputs['name'])
    assert outputs['name'] == 'SolarPlantRTO'
    assert outputs['x_lb'] == [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
    assert outputs['x_ub'] == [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
    ws = wb.sheets[0]
finally:
    # Save and close
    wb.save()
    wb.close()
    pass

In [ ]:
class MSExcelOptProblem(ConstrainedScalarOptimizationProblem):

    def __init__(self, filepath, cell_refs, sheet=1, global_minimum=None):
        self.filepath = filepath
        self.cell_refs = cell_refs
        self.sheet = sheet
        self._wb = None
        name = None
        bounds = None
        super().__init__(bounds, name=name, global_minimum=None)

    @property
    def name(self) -> str:
        name = get_var_value(self._ws, "name", self.cell_refs["name"])
        return name

    @property
    def bounds(self):
        lower_bounds = get_var_value(self._ws, "x_lb", self.cell_refs["x_lb"])
        upper_bounds = get_var_value(self._ws, "x_ub", self.cell_refs["x_ub"])
        return np.stack([lower_bounds, upper_bounds]).T

    @bounds.setter
    def bounds(self, value: np.ndarray) -> None:
        raise AttributeError(
            "Cannot set bounds using this method. Set them in the spreadsheet."
        )

    def __enter__(self):
        self._wb = xw.Book(self.filepath)
        self._ws = self._wb.sheets[self.sheet - 1]  # xlwings uses 0-based indexing
        self._bounds = self.bounds
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._wb.save()
        self._wb.close()

    def cost_function_to_minimize(self, x) -> float:
        inputs = {'x': x}
        outputs = evaluate_excel_sheet(
            self._wb, inputs, self.cell_refs, output_vars=['f(x)'], sheet=self.sheet
        )
        cost = outputs['f(x)']
        return cost
    
    # TODO: Implement constraint function

In [ ]:
# Test
with MSExcelOptProblem(filepath, cell_refs) as problem:
    print(problem.name)
    print(problem.bounds)
    x = [
        0.5,
        0.917004705,
        0.886780974,
        0.853623916,
        0.826111695,
        0.801142972,
        0.783225649,
        0.760051185,
        0.900587117,
        0.884820289,
        0.85561633,
        0.827130134,
        0.797646554,
        0.78549898,
        0.755400025
    ]
    print(problem(x))


In [ ]:
cell_refs = {
    'name': ((2, 2), (2, 3)),
    'f(x)': ((6, 13), (6, 14)),
    'x': (
        (5, 8), 
        [
            (6, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), 
            (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8)
        ]
    ),
    'g(x)': None,
    'x_lb': (
        (5, 9), 
        [
            (6, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), 
            (16, 9), (17, 9), (18, 9), (19, 9), (20, 9), (21, 9), (22, 9)
        ]
    ),
    'x_ub': (
        (5, 10), 
        [
            (6, 10), (9, 10), (10, 10), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), 
            (16, 10), (17, 10), (18, 10), (19, 10), (20, 10), (21, 10), (22, 10)
        ]
    )
}

class SolarPlantRTO(MSExcelOptProblem):

    def __init__(self, filepath):
        super().__init__(filepath, cell_refs)


with SolarPlantRTO(filepath) as problem:
    print(problem.name)
    print(problem.bounds.shape)
    inputs = {'x': [2.0]}
    outputs = evaluate_excel_sheet(
        problem._wb, inputs, problem.cell_refs, output_vars=['f(x)']
    )


In [ ]:
# Path to Excel file
problems_dir = 'problems'
problem_name = 'solar_plant_rto'
excel_filename = 'Solar Plant Optimization of N-Pumps I-O 2025-08-29.xlsm'
filepath = os.path.join(os.getcwd(), problems_dir, problem_name, excel_filename)

with SolarPlantRTO(filepath) as problem:
    sol = solve_problem_with_optimizer(
        problem, 
        lpfgopt.minimize, 
        problem.bounds, 
        points=15*5, 
        maxit=2, 
        tol=0.01
    )

In [ ]:
with problem as p:
    function_evaluations_plot(p)
plt.tight_layout()
plt.show()

In [ ]:
with problem as p:
    best_guesses_plot(p)
plt.tight_layout()
plt.show()

In [ ]:
f_best, x_best = problem.best_guess
x_best = [float(xi) for xi in x]
f_best, x_best

In [ ]:
with SolarPlantRTO(filepath) as problem:
    sol = solve_problem_with_optimizer(
        problem, 
        scipy.optimize.minimize, 
        x0=x_best, 
        bounds=problem.bounds, 
        tol=0.01, 
        options={'maxiter': 500}
    )

In [ ]:
with problem as p:
    best_guesses_plot(p)
plt.tight_layout()
plt.show()

In [ ]:
STOP HERE!!!

In [ ]:
fun_evals, unique_solutions, best_guesses = solve_problem_with_optimizer_n_repeats(problem, lpfgopt.minimize, 20, problem.bounds, tol=0.01)
unique_solutions

In [ ]:
ax = best_guesses_plot_n_repeats(fun_evals)
plt.tight_layout()
plt.show()

In [ ]:
min(best_guesses)

## Bayesian Optimization

In [ ]:
# Run Bayesian optimization
problem.reset()
res = skopt.gp_minimize(
    problem,            # the function to minimize
    problem.bounds,     # the bounds on each dimension of x
    n_calls=50,
    noise=1e-10,
    random_state=0,
    n_initial_points=10,
)
res

In [ ]:
res['x'], res['fun']

In [ ]:
problem.best_guess

In [ ]:
function_evaluations_plot(problem, markersize=5, linestyle='-')
plt.tight_layout()
plt.show()

In [ ]:
fun_evals, unique_solutions, best_guesses = solve_problem_with_optimizer_n_repeats(
    problem, skopt.gp_minimize, 10, problem.bounds, noise=1e-10, n_calls=50
)
unique_solutions

In [ ]:
ax = best_guesses_plot_n_repeats(fun_evals)
plt.tight_layout()
plt.show()

In [ ]:
min(best_guesses)